# Optimization Algorithms
Deep learning algorithms require optimizations since training very large deep neural network can be extremely slow. Other than applying a good initialization, batch normalization; a huge boost comes from using a faster optimization algorithm.

Here, three optimization algorithms will be presented.
- Gradient Descent 
- Momentum
- Adam 

### Gradient Descent

Gradient Descent is a primary optimization method in machine learning. 
For each layer l , the during forward propagation we calculate  W[l] and then during back propagation , we calculate derivatives of that respective weights and biases dW[l] and db[l].

For  l=1,...,L , the update is 

    W[l] = W[l] − α dW[l]
 
    b[l] = b[l] − α db[l]
 
where  α  is the learning rate.
All parameters and gradients should be stored in the parameters and gradients dictionary. 


In [14]:
def gradient_descent_update(parameters, gradients, learning_rate):

    L = len(parameters) 
    
    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * gradients['dW' + str(l + 1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * gradients['db' + str(l + 1)]
      
    return parameters

### Momentum

Momentum takes into account the past gradients to smooth out the update. because the path taken by mini-batch gradient descent will "oscillate" toward convergence. Using momentum can reduce these oscillations. We will store the 'direction' of the previous gradients in the variable  v . Formally, this will be the exponentially weighted average of the gradient on previous steps. 

<img src = "GD_M.jpg">
     
     
Figure: Difference between Gradient Descent and Momentum on mini batches.

So, at first, we initialize the velocity v, a python dictionary with arrays of zeros of the same size as of parameters W and b.
Then, we implement the parameters update with momentum. The momentum update rule for  l=1,...,L is 

        vdW[l]=βvdW[l]+(1−β)dW[l]
        W[l]=W[l]−αvdW[l]
 
        vdb[l]=βvdb[l]+(1−β)db[l]
        b[l]=b[l]−αvdb[l]
        
Where β is the momentum hyperparameter and  α is the learning rate. All parameters should be stored in the parameters dictionary.

In [12]:
def initialize_velocity(parameters):
    
    L = len(parameters) 
    v = {}
    
    for l in range(L):
        v["dW" + str(l+1)] = np.zeros_like(parameters["W" + str(l+1)])
        v["db" + str(l+1)] = np.zeros_like(parameters["b" + str(l+1)])
        
    return v

In [15]:
def momentum_update(parameters, gradients, v, beta, learning_rate):

    L = len(parameters) 

    for l in range(L):
        v["dW" + str(l+1)] = beta * v["dW" + str(l+1)] + ( 1- beta)*gradients['dW' + str(l+1)]
        v["db" + str(l+1)] = beta * v["db" + str(l+1)] + ( 1- beta)*gradients['db' + str(l+1)]
       
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * gradients['dW' + str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * gradients['db' + str(l+1)]
        
    return parameters, v

When β=0, it becomes standard gradient descent without momentum.

The larger the momentum  β is, the smoother the update because the more we take the past gradients into account. But if β is too big, it could also smooth out the updates too much.

Common values for  β  range from 0.8 to 0.999. 

### Adam

The most effective optimization algorithms for training neural networks is Adam. It combines ideas from RMSProp (described in lecture) and Momentum. The name Adam is derived from adaptive moment estimation.

It calculates an exponentially weighted average of past gradients, and stores it in variables v before bias correction and  vcorrected with bias correction. It then calculates an exponentially weighted average of the squares of the past gradients, and stores it in variables s before bias correction and  scorrected with bias correction.
It updates parameters in a direction based on combining information from v and s.
The update rule is, for  l=1,...,L

$$\begin{cases}
v_{dW^{[l]}} = \beta_1 v_{dW^{[l]}} + (1 - \beta_1) \frac{\partial \mathcal{J} }{ \partial W^{[l]} } \\
v^{corrected}_{dW^{[l]}} = \frac{v_{dW^{[l]}}}{1 - (\beta_1)^t} \\
s_{dW^{[l]}} = \beta_2 s_{dW^{[l]}} + (1 - \beta_2) (\frac{\partial \mathcal{J} }{\partial W^{[l]} })^2 \\
s^{corrected}_{dW^{[l]}} = \frac{s_{dW^{[l]}}}{1 - (\beta_2)^t} \\
W^{[l]} = W^{[l]} - \alpha \frac{v^{corrected}_{dW^{[l]}}}{\sqrt{s^{corrected}_{dW^{[l]}}} + \varepsilon}
\end{cases}$$

where:
t is the number of steps taken of Adam, L is the number of layers, β1 and β2 are hyperparameters that control the two exponentially weighted averages, α is the learning rate, ε  is a very small number to avoid dividing by zero.
As usual, we will store all parameters in the parameters dictionary. At first, we initialize the Adam variables v,s which keep track of the past information. The variables v,s are python dictionaries that need to be initialized with arrays of zeros. 

In [10]:
def initialize_adam(parameters) :
      
    L = len(parameters) 
    v = {}
    s = {}
    
    for l in range(L):
        v["dW" + str(l+1)] = np.zeros_like(parameters["W" + str(l + 1)])
        v["db" + str(l+1)] = np.zeros_like(parameters["b" + str(l + 1)])
        s["dW" + str(l+1)] = np.zeros_like(parameters["W" + str(l + 1)])
        s["db" + str(l+1)] = np.zeros_like(parameters["b" + str(l + 1)])

    return v, s

In [11]:
def adam_update(parameters, gradients, v, s, t, learning_rate = 0.01,
                                beta1 = 0.9, beta2 = 0.999,  epsilon = 1e-8):
  
    L = len(parameters)              
    v_corrected = {}                      
    s_corrected = {}                        
    
    for l in range(L):
      
        v["dW" + str(l+1)] = beta1 * v["dW" + str(l + 1)] + (1 - beta1) * gradients['dW' + str(l + 1)]
        v["db" + str(l+1)] = beta1 * v["db" + str(l + 1)] + (1 - beta1) * gradients['db' + str(l + 1)]
      
      
        v_corrected["dW" + str(l+1)] = v["dW" + str(l + 1)] / (1 - beta1**t)
        v_corrected["db" + str(l+1)] = v["dW" + str(l + 1)] / (1 - beta1**t)
      
        s["dW" + str(l+1)] =  beta2 * s["dW" + str(l + 1)] + (1 - beta2) *gradients['dW' + str(l + 1)]**2
        s["db" + str(l+1)] =  beta2 * s["db" + str(l + 1)] + (1 - beta2) * gradients['db' + str(l + 1)]**2
      

        s_corrected["dW" + str(l+1)] = s["dW" + str(l + 1)] / (1 - beta2**t)
        s_corrected["db" + str(l+1)] = s["dW" + str(l + 1)] / (1 - beta2**t)
      

        parameters["W" + str(l+1)] = parameters["W" + str(l + 1)] - learning_rate * v_corrected["dW" + str(l + 1)] / (np.sqrt(s_corrected["dW" + str(l + 1)]) + epsilon)
        parameters["b" + str(l+1)] = parameters["b" + str(l + 1)] - learning_rate * v_corrected["db" + str(l + 1)] / (np.sqrt(s_corrected["db" + str(l + 1)]) + epsilon)
       
    return parameters, v, s